In [1]:
from pyspark.sql import SparkSession
import pandas as pd 
import numpy as np
spark = SparkSession.builder \
    .appName("RedshiftAnalysis") \
    .getOrCreate()

In [26]:
import os 

In [2]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/home/jovyan/work/clients.csv")

In [3]:
from pyspark.sql import functions as F
df_clean = df.withColumn(
    'age_clean',
    F.when(F.col('age').isNull() | F.isnan(F.col('age')) | (F.col('age') == 'NaN'),
          F.lit(0))
     .otherwise(F.col('age').cast('double'))
)


In [4]:
age_moyen_df = df_clean.select(F.avg('age_clean').alias('age_moyen'))
#extraire la valeur de l'age moyen
ageMoyen = age_moyen_df.collect()[0][0]

In [5]:
df_clean = df_clean.withColumn('ageMoyen', F.lit(ageMoyen))

In [6]:
df_clean.show()

+---+------------+----+--------------------+-------+---------+---------+--------+
| id|         nom| age|               email| revenu|    ville|age_clean|ageMoyen|
+---+------------+----+--------------------+-------+---------+---------+--------+
|  1| Jean Dupont|25.0|jean.dupont@email...|35000.0|    Paris|     25.0|    24.7|
|  2|        NULL|30.0|martine.lenoir@em...|42000.0|     Lyon|     30.0|    24.7|
|  3|     Ali Ben| NaN|   ali.ben@email.com|50000.0|Marseille|      0.0|    24.7|
|  4| Marie Curie|40.0|marie.curie@email...|   NULL|      NaN|     40.0|    24.7|
|  5|         NaN|28.0|thomas.muller@ema...|38000.0| Bordeaux|     28.0|    24.7|
|  6| Paul Martin|NULL|paul.martin@email...|    NaN|    Lille|      0.0|    24.7|
|  7|Clara Durand|35.0|clara.durand@emai...|48000.0|      NaN|     35.0|    24.7|
|  8| Lucas Morel|22.0|   lucas.morel@email|32000.0|     Nice|     22.0|    24.7|
|  9|  Sophie NaN|38.0|sophie.blanche@em...|51000.0|   Rennes|     38.0|    24.7|
| 10|  Marc Peti

In [25]:
# Enregistrer le dataframe pyspark dans aws redshift 

In [ ]:
REDSHIFT_USER = ''
REDSHIFT_PASSWORD = ''
# Format correct pour Redshift avec le pilote PostgreSQL
REDSHIFT_FULL_PATH = "jdbc:postgresql://workgroupe./././.amazonaws.com:port/db"
REDSHIFT_TABLE = 'clients'

In [ ]:
mode = "overwrite"
properties = {
    "user": REDSHIFT_USER, 
    "password": REDSHIFT_PASSWORD, 
    "driver": "org.postgresql.Driver"
}
df_clean.write.jdbc(url=REDSHIFT_FULL_PATH, table=REDSHIFT_TABLE, mode=mode, properties=properties)